# Import Necessary Packages

In [2]:
import tensorflow as tf
import numpy as np
import os,random,cv2               
import matplotlib.pyplot as plt    # For Plotting and viewing images

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
start = 0

# Reading Training and Testing Dataset

In [ ]:
def listFiles(path):
    dirs = os.listdir(path)
    filelists = []
    for i in dirs:
        filelists.append(os.path.join(path,i))        
    return filelists

def read_dataset(filelists,batch_size):
    imagedata = np.zeros((batch_size,128,128,3),dtype=np.float32)                         
    labels = np.zeros((batch_size,2),dtype=np.float32)
    global start
    
    if (start+1)*batch_size > len(filelists):
        random.shuffle(filelists)
        start = 0
        
    for i in range(start*batch_size,(start+1)*batch_size):
        img = cv2.imread(filelists[i])
        img = np.resize(img,(128,128,3))
        img = np.reshape(img,(-1,128,128,3))

        index = filelists[i].find("cat")
        if(index!=-1):
            label = [1.0,0.0]  # CAT  DOG
        else:
            label = [0.0,1.0]
        
        imagedata[i-start*batch_size]=img
        labels[i-start*batch_size]=label
        
    start = start+1   
    return imagedata.reshape((batch_size,49152)),labels.reshape((batch_size,2))
        

# Defining CONV and FC Layers

In [ ]:
def conv_layer(input_img,size_in,size_out,name="conv"):
    W = tf.Variable(tf.random_normal([3,3,size_in,size_out]),dtype=tf.float32,name="Weight")
    b = tf.Variable(tf.constant([size_out],tf.float32,name="Bias"))
    f_map = tf.nn.conv2d(input_img,W,strides=[1,1,1,1],padding="SAME")
    f_map2 = tf.nn.relu(f_map+b)
    return tf.nn.max_pool(f_map2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

def fc_layer(input_img,size_in,size_out,name="fc"):
    W = tf.Variable(tf.random_normal([size_in,size_out]),dtype=tf.float32,name="Weight")
    b = tf.Variable(tf.constant([size_out],tf.float32,name="Bias"))
    #In[0]: [32768,2], In[1]: [100,32768]
    
    f_map = tf.matmul(input_img,W)+b    
    f_map2 = tf.nn.relu(f_map)    
    return f_map2

Iteration 0 is running.
Iteration 1 is running.
Iteration 2 is running.
Iteration 3 is running.
Iteration 4 is running.
Iteration 5 is running.
Iteration 6 is running.
Iteration 7 is running.
Iteration 8 is running.
Iteration 9 is running.
Iteration 10 is running.
Iteration 11 is running.
Iteration 12 is running.
Iteration 13 is running.
Iteration 14 is running.
Iteration 15 is running.
Iteration 16 is running.
Iteration 17 is running.
Iteration 18 is running.
Iteration 19 is running.
Iteration 20 is running.
Iteration 21 is running.
Iteration 22 is running.
Iteration 23 is running.
Iteration 24 is running.
Iteration 25 is running.
Iteration 26 is running.
Iteration 27 is running.
Iteration 28 is running.
Iteration 29 is running.
Iteration 30 is running.
Iteration 31 is running.
Iteration 32 is running.


# Creating a CNN Architecture

In [ ]:
def cnn_model():
    x = tf.placeholder(tf.float32,(None,49152))
    x_image = tf.reshape(x,[-1,128,128,3])
    y_true = tf.placeholder(tf.float32,(None,2)) # one-hot encoding
    
    conv1 = conv_layer(x_image,3,16,"conv1")
    conv2 = conv_layer(conv1,16,32,"conv2")
    
    
    (d,a,b,c) = conv2.shape
    flatten = tf.reshape(conv2,[-1,a*b*c])
    
    fc1 = fc_layer(flatten,32768,2,"fc1")
    
    xent = tf.nn.softmax_cross_entropy_with_logits(logits=fc1,labels=y_true)
    loss = tf.reduce_mean(xent)    
    train = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    correct_pred = tf.equal(tf.argmax(y_true,1),tf.argmax(fc1,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    
    filelists = listFiles("./train/")
    for i in range(250*50):
        print("Iteration "+str(i)+" is running.")
        x_train,y_train = read_dataset(filelists,100)
        sess.run(train,feed_dict={x:x_train, y_true: y_train})
    

        
        
    filelists = listFiles("./train/")
    start = 0  
    x_test,y_test = read_dataset(filelists,len(filelists))
    accuracy = sess.run(accuracy,feed_dict={x:x_test,y_true:y_test})
    
    print("Accuracy is %f"%(accuracy)) 
    
    

# Executing and Testing Accuracy

In [ ]:
cnn_model()